# 使用Langchain、AnalyticDB和OpenAI进行问答
本笔记本展示了如何使用Langchain、AnalyticDB作为知识库和OpenAI嵌入来实现一个问答系统。如果您对AnalyticDB不熟悉，最好先查看[Getting_started_with_AnalyticDB_and_OpenAI.ipynb](Getting_started_with_AnalyticDB_and_OpenAI.ipynb)笔记。

本笔记本展示了一个端到端的流程：
- 使用OpenAI API计算嵌入。
- 将嵌入存储在AnalyticDB实例中以构建知识库。
- 将原始文本查询转换为嵌入，使用OpenAI API。
- 使用AnalyticDB在创建的集合中执行最近邻搜索，以找到一些上下文。
- 请求LLM在给定上下文中找到答案。

所有步骤将简化为调用一些相应的Langchain方法。


## 先决条件
为了完成这个练习，我们需要准备一些东西：
- [AnalyticDB 云实例](https://www.alibabacloud.com/help/en/analyticdb-for-postgresql/latest/product-introduction-overview)。
- [Langchain](https://github.com/hwchase17/langchain) 作为框架。
- OpenAI API 密钥。


### 安装要求
此笔记本需要以下Python包：`openai`、`tiktoken`、`langchain`和`psycopg2cffi`。
- `openai` 提供了方便访问OpenAI API的功能。
- `tiktoken` 是一个快速的BPE标记器，可与OpenAI的模型一起使用。
- `langchain` 帮助我们更轻松地构建具有LLM的应用程序。
- `psycopg2cffi` 库用于与向量数据库交互，但也可以使用任何其他PostgreSQL客户端库。


In [ ]:
! pip install openai tiktoken langchain psycopg2cffi 


In [2]:
! export OPENAI_API_KEY="your API key"


In [1]:
# 测试您的OpenAI API密钥是否已正确设置为环境变量。
# 注意：如果您在本地运行此笔记本，您需要重新加载终端和笔记本，以使环境变量生效。
import os

# 注意：或者，您也可以像这样设置一个临时的环境变量：
# os.environ["OPENAI_API_KEY"] = "sk-xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"

if os.getenv("OPENAI_API_KEY") is not None:
    print("OPENAI_API_KEY is ready")
else:
    print("OPENAI_API_KEY environment variable not found")


OPENAI_API_KEY is ready


### 准备您的OpenAI API密钥
OpenAI API密钥用于对文档和查询进行向量化。

如果您没有OpenAI API密钥，可以从[https://platform.openai.com/account/api-keys](https://platform.openai.com/account/api-keys)获取。

获取密钥后，请通过运行以下命令将其添加到您的环境变量中，命名为`OPENAI_API_KEY`：


### 准备您的 AnalyticDB 连接字符串
要构建 AnalyticDB 连接字符串，您需要拥有以下参数：`PG_HOST`、`PG_PORT`、`PG_DATABASE`、`PG_USER` 和 `PG_PASSWORD`。您需要首先导出它们以设置正确的连接字符串。然后构建连接字符串。


In [4]:
! export PG_HOST="your AnalyticDB host url"
! export PG_PORT=5432 # 可选，默认值为 5432
! export PG_DATABASE=postgres # 可选，默认值为 postgres
! export PG_USER="your username"
! export PG_PASSWORD="your password"


In [2]:
import os
from langchain.vectorstores.analyticdb import AnalyticDB

CONNECTION_STRING = AnalyticDB.connection_string_from_db_params(
    driver=os.environ.get("PG_DRIVER", "psycopg2cffi"),
    host=os.environ.get("PG_HOST", "localhost"),
    port=int(os.environ.get("PG_PORT", "5432")),
    database=os.environ.get("PG_DATABASE", "postgres"),
    user=os.environ.get("PG_USER", "postgres"),
    password=os.environ.get("PG_PASSWORD", "postgres"),
)


In [3]:
import json

with open("questions.json", "r") as fp:
    questions = json.load(fp)

with open("answers.json", "r") as fp:
    answers = json.load(fp)


## 加载数据
在这一部分，我们将加载包含一些自然问题及其答案的数据。所有数据将用于创建一个以AnalyticDB为知识库的Langchain应用程序。


In [4]:
print(questions[0])


when is the last episode of season 8 of the walking dead


In [ ]:
import wget

# 所有示例均来自 https://ai.google.com/research/NaturalQuestions。
# 这是我们从下载并提取的训练集中抽取的一个样本，用于某些特定目的。
# 进一步加工。
wget.download("https://storage.googleapis.com/dataset-natural-questions/questions.json")
wget.download("https://storage.googleapis.com/dataset-natural-questions/answers.json")


In [6]:
print(answers[0])


No . overall No. in season Title Directed by Written by Original air date U.S. viewers ( millions ) 100 `` Mercy '' Greg Nicotero Scott M. Gimple October 22 , 2017 ( 2017 - 10 - 22 ) 11.44 Rick , Maggie , and Ezekiel rally their communities together to take down Negan . Gregory attempts to have the Hilltop residents side with Negan , but they all firmly stand behind Maggie . The group attacks the Sanctuary , taking down its fences and flooding the compound with walkers . With the Sanctuary defaced , everyone leaves except Gabriel , who reluctantly stays to save Gregory , but is left behind when Gregory abandons him . Surrounded by walkers , Gabriel hides in a trailer , where he is trapped inside with Negan . 101 `` The Damned '' Rosemary Rodriguez Matthew Negrete & Channing Powell October 29 , 2017 ( 2017 - 10 - 29 ) 8.92 Rick 's forces split into separate parties to attack several of the Saviors ' outposts , during which many members of the group are killed ; Eric is critically injure

## 链定义

Langchain已经集成了AnalyticDB，并对给定的文档列表执行所有索引。在我们的案例中，我们将存储我们拥有的一组答案。


In [7]:
from langchain.vectorstores import AnalyticDB
from langchain.embeddings import OpenAIEmbeddings
from langchain import VectorDBQA, OpenAI

embeddings = OpenAIEmbeddings()
doc_store = AnalyticDB.from_texts(
    texts=answers, embedding=embeddings, connection_string=CONNECTION_STRING,
    pre_delete_collection=True,
)


在这个阶段，所有可能的答案都已经存储在AnalyticDB中，因此我们可以定义整个问答链条。


In [9]:
from langchain.chains import RetrievalQA

llm = OpenAI()
qa = VectorDBQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    vectorstore=doc_store,
    return_source_documents=False,
)


## 搜索数据

一旦数据被放入AnalyticDB中，我们就可以开始提出一些问题。一个问题将会被OpenAI模型自动转换为向量，然后使用创建的向量在AnalyticDB中找到一些可能匹配的答案。一旦检索到，最相似的答案将被合并到发送给OpenAI大型语言模型的提示中。


In [10]:
import random

random.seed(52)
selected_questions = random.choices(questions, k=5)


In [11]:
for question in selected_questions:
    print(">", question)
    print(qa.run(question), end="\n\n")


> where do frankenstein and the monster first meet
 Victor retreats into the mountains, and that is where the Creature finds him and pleads for Victor to hear his tale.

> who are the actors in fast and furious
 The main cast of Fast & Furious includes Vin Diesel as Dominic Toretto, Paul Walker as Brian O'Conner, Michelle Rodriguez as Letty Ortiz, Jordana Brewster as Mia Toretto, Tyrese Gibson as Roman Pearce, and Ludacris as Tej Parker.

> properties of red black tree in data structure
 The properties of a red-black tree in data structure are that each node is either red or black, the root is black, all leaves (NIL) are black, and if a node is red, then both its children are black. Additionally, every path from a given node to any of its descendant NIL nodes contains the same number of black nodes.

> who designed the national coat of arms of south africa
 Iaan Bekker

> caravaggio's death of the virgin pamela askew
 I don't know.



### 自定义提示模板

Langchain中的`stuff`链类型使用特定的提示，其中包含问题和上下文文档。这是默认提示的样式：

```text
使用以下上下文片段来回答最后的问题。如果你不知道答案，只需说你不知道，不要试图凭空编造答案。
{context}
问题：{question}
有用的答案：
```

然而，我们可以提供自定义的提示模板，并改变OpenAI LLM的行为，同时仍然使用`stuff`链类型。重要的是保持`{context}`和`{question}`作为占位符。

#### 尝试自定义提示

我们可以尝试使用不同的提示模板，这样模型：
1. 如果知道答案，就用一个简短的句子回答。
2. 如果不知道问题的答案，就建议一个随机的歌曲标题。


In [12]:
from langchain.prompts import PromptTemplate
custom_prompt = """
Use the following pieces of context to answer the question at the end. Please provide
a short single-sentence summary answer only. If you don't know the answer or if it's
not present in given context, don't try to make up an answer, but suggest me a random
unrelated song title I could listen to.
Context: {context}
Question: {question}
Helpful Answer:
"""

custom_prompt_template = PromptTemplate(
    template=custom_prompt, input_variables=["context", "question"]
)


In [13]:
custom_qa = VectorDBQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    vectorstore=doc_store,
    return_source_documents=False,
    chain_type_kwargs={"prompt": custom_prompt_template},
)


In [14]:
random.seed(41)
for question in random.choices(questions, k=5):
    print(">", question)
    print(custom_qa.run(question), end="\n\n")


> what was uncle jesse's original last name on full house
Uncle Jesse's original last name on Full House was Cochran.

> when did the volcano erupt in indonesia 2018
No information about a volcano erupting in Indonesia in 2018 is present in the given context. Suggested song title: "Volcano" by U2.

> what does a dualist way of thinking mean
A dualist way of thinking means believing that humans possess a non-physical mind or soul which is distinct from their physical body.

> the first civil service commission in india was set up on the basis of recommendation of
The first Civil Service Commission in India was not set up on the basis of a recommendation.

> how old do you have to be to get a tattoo in utah
In Utah, you must be at least 18 years old to get a tattoo.

